In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer #permite transformar palabras para que sea mas entendible (letras de mas o errores de tipeo, ect).
stemmer = LancasterStemmer ()
import numpy
import tensorflow
import tflearn
import json
import random
import pickle #permite guardar el modelo y no cargarlo cada vez que se ejecute el programa
import csv
import pandas as pd
import tkinter as tk

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
with open("estandar.json", encoding='utf-8') as archivo:
    datos = json.load(archivo)
    
# print(datos) #muestra de datos y prueba de carga correcta

In [4]:
from tensorflow.python.framework import ops

palabras=[]
tags=[]
auxX=[] #palabras que se asignaran a la lista "palabras"
auxY=[]

for contenido in datos["contenido"]:
    for patrones in contenido["patrones"]:
        auxPalabra = nltk.word_tokenize(patrones) # Toma una frase y la separa en palabras (tokens), para identificar los patrones a parametrizar
        palabras.extend(auxPalabra)               # Almacena todas las palabras tokenizadas 
        auxX.append(auxPalabra)                   
        auxY.append(contenido["tag"])

        if contenido["tag"] not in tags:          # Agrega un tag si no lo encuentra
            tags.append(contenido["tag"])

palabras = [stemmer.stem(w.lower()) for w in palabras if w != "\n"]   
palabras = sorted(list(set(palabras)))            # Se define la palabra
tags = sorted(tags)                               # Se define el tag

entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]

for x, documento in enumerate(auxX):              # en X se guardara el indice de la palabra
    cubeta=[]                                     #Lista auxiliar para almacenar las palabras 
    auxPalabra = [stemmer.stem(w.lower()) for w in documento]
    for w in palabras:
        if w in auxPalabra:
            cubeta.append(1)
        else:
            cubeta.append(0)

    filaSalida = salidaVacia[:]
    filaSalida[tags.index(auxY[x])] = 1            #en la posicion del indice del tag, obtenemos el indice del auxiliar
    entrenamiento.append(cubeta)
    salida.append(filaSalida)

entrenamiento = numpy.array(entrenamiento)
salida = numpy.array(salida)


In [5]:
ops.reset_default_graph()

    ### creacion red neuronal

#    modelo.load("modelo.tflearn")
#except:

#modelo.fit(entrenamiento,salida,n_epoch=1000,batch_size=40,show_metric=True) #Entrenamiento para el proceso de modelado
#modelo.save("modelo.tflearn")


In [19]:
from tkinter import messagebox
from tkinter.ttk import Progressbar

from time import sleep
from multiprocessing import Pool
from multiprocessing import Process, current_process

class Proceso(tk.Frame):

    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()

        self.inicializar_gui()

    def inicializar_gui(self):
        self.progreso = tk.DoubleVar()
        self.pbr_tarea = Progressbar(self, length=250, style='black.Horizontal.TProgressbar', variable=self.progreso, maximum=100)
        self.pbr_tarea['value'] = 0
        self.pbr_tarea.grid(row=0, column=0)

        tk.Button(self, text='Iniciar tarea', command=self.iniciar_tarea).grid(row=1, column=0)
    
    def iniciar_tarea(self):
        contador = 0
        
        red = tflearn.input_data(shape=[None,len(entrenamiento[0])]) # Datos de entrada a entrenar
        red = tflearn.fully_connected(red,10)                        # Se crean capas conectadas con 10 unidades/neuronas
        red = tflearn.fully_connected(red,10)
        red = tflearn.fully_connected(red,len(salida[0]),activation="softmax")
        red = tflearn.regression(red) #permite obtener la probabilidad de que tan eficaz es la clasificacion de los tag

        modelo = tflearn.DNN(red)
    #try:
    #    modelo.load("modelo.tflearn")
    #except:


        #n_epoch = 0
        while contador <= 200:
            self.progreso.set(contador)
            contador += 10
            
        def  fnc_square():
            modelo.fit(entrenamiento,salida,n_epoch=200,batch_size=40,show_metric=True) #Entrenamiento para el proceso de modelado
            modelo.save("modelo.tflearn")
    
        pool = Pool(processes=4)
        result = pool.apply_async(fnc_square)
        result.get(timeout=1)
        pool.map(fnc_square())
        
        
        self.master.update_idletasks()
        sleep(1)

        messagebox.showinfo('Mensaje', 'La tarea ha terminado')


In [20]:
app = tk.Tk()
app.title('Proceso')
app.geometry('300x300')

proceso = Proceso(app)
proceso.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\smart\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\smart\AppData\Local\Temp\ipykernel_21316\846348274.py", line 51, in iniciar_tarea
    result.get(timeout=1)
  File "C:\Users\smart\anaconda3\lib\multiprocessing\pool.py", line 771, in get
    raise self._value
  File "C:\Users\smart\anaconda3\lib\multiprocessing\pool.py", line 537, in _handle_tasks
    put(task)
  File "C:\Users\smart\anaconda3\lib\multiprocessing\connection.py", line 211, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "C:\Users\smart\anaconda3\lib\multiprocessing\reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'Proceso.iniciar_tarea.<locals>.fnc_square'
